# Проверка орфографии при помощи трансформеров

In [1]:
import wandb
from src.datasets import load_datasets

orpho_dataset, punct_dataset = load_datasets()

c:\lang_check\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since ai-forever/spellcheck_benchmark couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'RUSpellRU' at C:\Users\Андрей Т\.cache\huggingface\datasets\ai-forever___spellcheck_benchmark\RUSpellRU\0.0.1\3395aa540689e4393c3e18d063e73a5b99d7f047 (last modified on Mon Jun 17 00:55:50 2024).


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from typing import List, Tuple, Dict


class TransformerSpellChecker:
    """
    Класс для проверки орфографии и пунктуации с использованием модели на основе Transformers.

    Attributes:
        tokenizer (AutoTokenizer): Токенизатор для модели.
        model (AutoModelForSeq2SeqLM): Модель для генерации исправленного текста.
    """

    def __init__(
        self,
        model_name: str = "ai-forever/sage-fredt5-distilled-95m",
        device: str = "cuda",
    ):
        """
        Инициализация токенизатора и модели.

        Args:
            model_name (str): Название предобученной модели. По умолчанию "ai-forever/sage-fredt5-distilled-95m".
            device (str): Устройство для вычислений ("cuda" или "cpu"). По умолчанию "cuda".
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.device = torch.device(device if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def predict_verbose(self, text: str) -> Tuple[List[Dict[str, str]], str]:
        """
        Возвращает исправленный текст и список всех предложенных исправлений.

        Args:
            text (str): Входной текст для проверки.

        Returns:
            Tuple[List[Dict[str, str]], str]: Исправленный текст и список всех предложенных исправлений.
        """
        inputs = self.tokenizer(
            text,
            max_length=None,
            padding="longest",
            truncation=False,
            return_tensors="pt",
        )
        outputs = self.model.generate(
            **inputs.to(self.model.device), max_length=inputs["input_ids"].size(1) * 1.5
        )
        corrected_text = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)[
            0
        ]

        corrections = self._generate_corrections(text, corrected_text)
        return corrections, corrected_text

    def _generate_corrections(
        self, original_text: str, corrected_text: str
    ) -> List[Dict[str, str]]:
        """
        Генерирует список исправлений на основе оригинального и исправленного текста.

        Args:
            original_text (str): Оригинальный текст.
            corrected_text (str): Исправленный текст.

        Returns:
            List[Dict[str, str]]: Список исправлений.
        """
        corrections = []
        original_words = original_text.split()
        corrected_words = corrected_text.split()

        # Идем по минимальной длине списков
        for idx, (original_word, corrected_word) in enumerate(
            zip(original_words, corrected_words)
        ):
            if original_word != corrected_word:
                correction = {
                    "index": original_text.index(original_word),
                    "error": original_word,
                    "suggestions": [corrected_word],
                    "message": "",
                }
                corrections.append(correction)

        # Если есть оставшиеся слова в оригинальном тексте
        if len(original_words) > len(corrected_words):
            for word in original_words[len(corrected_words) :]:
                correction = {
                    "index": original_text.index(word),
                    "error": word,
                    "suggestions": [],
                    "message": "",
                }
                corrections.append(correction)

        # Если есть добавленные слова в исправленном тексте
        if len(corrected_words) > len(original_words):
            for word in corrected_words[len(original_words) :]:
                correction = {
                    "index": len(original_text),
                    "error": "",
                    "suggestions": [word],
                    "message": "",
                }
                corrections.append(correction)

        return corrections

    def predict(self, text: str) -> str:
        """
        Возвращает исправленный текст.

        Args:
            text (str): Входной текст для проверки.

        Returns:
            str: Исправленный текст.
        """
        _, corrected_text = self.predict_verbose(text)
        return corrected_text

c:\lang_check\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
m = TransformerSpellChecker()

Задаем список моделей, которые будем тестировать (в порядке их приоритета)

In [3]:
PROJECT_NAME = "transformer_spellers"

MODEL_NAMES = [
    "sage-fredt5-distilled-95m",
    "sage-fredt5-large",
    "sage-m2m100-1.2B",
    "sage-mt5-large",
]
MODEL_NAMES = map(lambda x: f"ai-forever/{x}", MODEL_NAMES)

In [4]:
from typing import Literal
from src.model_scorers import WandbSageModelScorer


def _score_model(mode: str, dataset, model_name: str):
    sms = WandbSageModelScorer(dataset, project=PROJECT_NAME, run_suffix=mode)
    model = TransformerSpellChecker(model_name)
    scoring_final_result, explanation = sms.score_explain(
        model, metrics=["errant", "ruspelleval"]
    )
    print(f"{model_name} ({mode}):")
    print(scoring_final_result, explanation, sep="\n\n\n")
    return scoring_final_result, explanation


def score(mode: Literal["orpho"] | Literal["punct"], model_name):
    if mode == "orpho":
        _score_model(
            mode="orpho",
            dataset=orpho_dataset["test"],
            model_name=model_name,
        )
    elif mode == "punct":
        _score_model(
            mode="punct",
            dataset=punct_dataset["test"],
            model_name=model_name,
        )
    else:
        raise ValueError("No such mode")

# Тестирование: орфография и пунктуация

Эксперимент до конца не завершён - "железа" не хватило на все модели

Будет перезапущен на Google Colab (GPU)

In [5]:
for model_name in MODEL_NAMES:
    score("orpho", model_name)
    score("punct", model_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: andtun (obuchii). Use `wandb login --relogin` to force relogin


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Calculating words metric: 100%|██████████| 2008/2008 [00:06<00:00, 316.56it/s]


ai-forever/sage-fredt5-distilled-95m (orpho):
{'SPELL_Precision': 64.74, 'SPELL_Recall': 68.91, 'SPELL_F1': 66.76, 'PUNCT_Precision': 0.02, 'PUNCT_Recall': 33.33, 'PUNCT_F1': 0.04, 'CASE_Precision': 0.0, 'CASE_Recall': 100.0, 'CASE_F1': 0.0, 'YO_Precision': 0.0, 'YO_Recall': 100.0, 'YO_F1': 0.0, 'Precision': 82.19, 'Recall': 77.25, 'F1': 79.65}


                                                 Source  \
0        ﻿есть у вас оформленый и подписаный мною заказ   
1     вот в инете откапал такую интеерсную статейку ...   
2     я на всю жизнь запомню свое первое купание в з...   
3     думаем что не ошибемся если скажем что выставк...   
4     судьба человека может складываться очень разно...   
...                                                 ...   
2003  спасибо вам огромное за нормальную новость о е...   
2004  более захватывающее и наглядное обучение возмо...   
2005  и вобщем-то все понятно на фоне слухов застави...   
2006                               но всему есть придел   
20

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Calculating words metric: 100%|██████████| 2008/2008 [00:05<00:00, 348.75it/s]


ai-forever/sage-fredt5-distilled-95m (punct):
{'CASE_Precision': 94.37, 'CASE_Recall': 92.55, 'CASE_F1': 93.45, 'SPELL_Precision': 77.49, 'SPELL_Recall': 64.09, 'SPELL_F1': 70.15, 'PUNCT_Precision': 86.78, 'PUNCT_Recall': 80.57, 'PUNCT_F1': 83.56, 'YO_Precision': 46.21, 'YO_Recall': 73.83, 'YO_F1': 56.84, 'Precision': 83.43, 'Recall': 74.75, 'F1': 78.85}


                                                 Source  \
0     а так хочеться что-то мочь менять в этом мире ...   
1     давольно милый и летом и зимой обогреваемый те...   
2     бывают такие моменты когда хочеться зделать чт...   
3        ﻿есть у вас оформленый и подписаный мною заказ   
4     вот в инете откапал такую интеерсную статейку ...   
...                                                 ...   
2003  спасибо вам огромное за нормальную новость о е...   
2004  более захватывающее и наглядное обучение возмо...   
2005  и вобщем-то все понятно на фоне слухов застави...   
2006                               но всему есть пр

...

ai-forever/sage-fredt5-large (orpho):
{'SPELL_Precision': 52.07, 'SPELL_Recall': 67.5, 'SPELL_F1': 58.79, 'CASE_Precision': 0.0, 'CASE_Recall': 100.0, 'CASE_F1': 0.0, 'PUNCT_Precision': 0.03, 'PUNCT_Recall': 33.33, 'PUNCT_F1': 0.06, 'YO_Precision': 0.0, 'YO_Recall': 100.0, 'YO_F1': 0.0, 'Precision': 56.75, 'Recall': 70.58, 'F1': 62.91}


                                                 Source  \
0        ﻿есть у вас оформленый и подписаный мною заказ   
1     вот в инете откапал такую интеерсную статейку ...   
2     я на всю жизнь запомню свое первое купание в з...   
3     думаем что не ошибемся если скажем что выставк...   
4     судьба человека может складываться очень разно...   
...                                                 ...   
2003  спасибо вам огромное за нормальную новость о е...   
2004  более захватывающее и наглядное обучение возмо...   
2005  и вобщем-то все понятно на фоне слухов застави...   
2006                               но всему есть придел   
2007  у нас

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Calculating words metric: 100%|██████████| 2008/2008 [00:08<00:00, 249.81it/s]


ai-forever/sage-fredt5-large (punct):
{'CASE_Precision': 92.08, 'CASE_Recall': 67.82, 'CASE_F1': 78.11, 'SPELL_Precision': 55.2, 'SPELL_Recall': 55.59, 'SPELL_F1': 55.39, 'PUNCT_Precision': 86.69, 'PUNCT_Recall': 46.15, 'PUNCT_F1': 60.23, 'YO_Precision': 0.0, 'YO_Recall': 0.0, 'YO_F1': 0.0, 'Precision': 57.44, 'Recall': 68.1, 'F1': 62.32}


                                                 Source  \
0     а так хочеться что-то мочь менять в этом мире ...   
1     давольно милый и летом и зимой обогреваемый те...   
2     бывают такие моменты когда хочеться зделать чт...   
3        ﻿есть у вас оформленый и подписаный мною заказ   
4     вот в инете откапал такую интеерсную статейку ...   
...                                                 ...   
2003  спасибо вам огромное за нормальную новость о е...   
2004  более захватывающее и наглядное обучение возмо...   
2005  и вобщем-то все понятно на фоне слухов застави...   
2006                               но всему есть придел   
2007  у 

c:\lang_check\.venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Андрей Т\.cache\huggingface\hub\models--ai-forever--sage-m2m100-1.2B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


OSError: Unable to load vocabulary from file. Please check that the provided vocabulary is accessible and not corrupted.

In [ ]:
wandb.run.finish()

AttributeError: 'NoneType' object has no attribute 'finish'

# Выводы

### ai-forever/sage-fredt5-distilled-95m

https://wandb.ai/obuchii/transformer_spellers/runs/cj4hf61p

|  Задача | Errant Precision | Errant Recall | Errant F1 | Precision | Recall | F1
|----------|----------|----------|----------|----------|----------|----------|
| Spelling   | 64.74   | 68.91   | 66.76    | 82.19   | 77.25   | 79.65   |

-------


 https://wandb.ai/obuchii/transformer_spellers/runs/21macwva
 
|  Задача | Punct Precision | Punct Recall | Punct F1 | Case Precision | Case Recall | Case F1
|----------|----------|----------|----------|----------|----------|----------|
| Punctuation   | 86.78   | 80.57   | 83.56   |  94.37  | 92.55   | 93.45   |




### ai-forever/sage-fredt5-large

https://wandb.ai/obuchii/transformer_spellers/runs/w4g1na9g

|  Задача | Errant Precision | Errant Recall | Errant F1 | Precision | Recall | F1
|----------|----------|----------|----------|----------|----------|----------|
| Spelling   | 52.07   | 67.5   | 58.79   | 56.75  | 70.58  | 62.91    |

-------

  https://wandb.ai/obuchii/transformer_spellers/runs/7ojvllsk

 
|  Задача | Punct Precision | Punct Recall | Punct F1 | Case Precision | Case Recall | Case F1
|----------|----------|----------|----------|----------|----------|----------|
| Punctuation   | 86.69   | 46.15  | 60.23    | 92.08  | 67.82   |  78.11    |


# Итог

Лучше всего себя показала ai-forever/sage-fredt5-distilled-95m, с хорошей точностью исправляющая и орфографию, и пунктуацию

Она доступна на локально поднимаемом сервере (см README в корневом каталоге)